In [275]:
import pandas as pd

In [276]:
# Import a dataset containing a unique gameid and its name
Names = pd.read_csv('game_names.csv')


In [278]:
# Now import the twitch data. Twitch data contains games that are not contained in prize data and vice versa.
# Thus when we merge them, we will only use ones that shows up in both data. 
twitch = pd.read_csv('twitch.csv')
data = pd.merge(Names, twitch, on='GameName', how='inner')

In [279]:
# Now twitch data is cleaned. We will clean country_lang dataset

language = pd.read_csv('country_lang.csv')

In [280]:
# While scraping, we scraped something that are relavant, and they have shown up as missing value, so we will drop them.
language = language.dropna()

In [281]:
# Language value contains comma and also sometimes it contains multiple language. We will pick the first one and drop the rest. 
def string(text):
    text = str(text)
    return text.split(',')[0]

language['language'] = language['language'].apply(string)

In [282]:
# Now that we have cleaned the country_lang dataset, we can attach a language to the country data contained in broadband data. 

broad = pd.read_csv('broadband.csv')
broad = pd.merge(broad, language, left_on='Entity', right_on = 'country',how='inner')
broad.drop('country', axis=1)

# Now broadband data has language data attached to it. We want to make sure that language here is consistent with the language
# used in the twitch data. 

,Entity,Code,Year,Fixed broadband subscriptions (per 100 people),language
0,Afghanistan,AFG,2004,0.000849,Dari
1,Afghanistan,AFG,2005,0.000901,Dari
2,Afghanistan,AFG,2006,0.001965,Dari
3,Afghanistan,AFG,2007,0.001930,Dari
4,Afghanistan,AFG,2008,0.001892,Dari
...,...,...,...,...,...
3419,Zimbabwe,ZWE,2017,1.269804,English
3420,Zimbabwe,ZWE,2018,1.349014,English
3421,Zimbabwe,ZWE,2019,1.331353,English
3422,Zimbabwe,ZWE,2020,1.298439,English


In [283]:
# To have a consistent language, take the symmetric difference of language sets to see what language is lacking. 
twi_lan = set(twitch['language'].unique())
bro_lan = set(broad['language'].unique())
print(twi_lan - bro_lan)
print(bro_lan - twi_lan)

{'Spanish (Mexico)', 'American Sign Language', 'Chinese', 'Chinese (Hong Kong SAR)', 'Czech', 'Greek', 'Malaysian', 'Polish', 'Other', 'Hungarians', 'Catalans', 'Portuguese (Brazil)', 'Chinese (Taiwan)', 'Italians', 'All languages'}
{'Hungarian', 'Hebrew', 'Armenian', 'Lithuanian', 'Serbian', 'Haitian', 'Uzbek', 'Montenegrin', 'Turkmen', 'Laotian', 'Samoan', 'Seychelles Creole', 'Kirundi', 'Icelandic', 'Estonian', 'Albanian', 'Tetum', 'Kazakh', 'Filipino', 'none', 'Ruanda', 'Amharic', 'Georgian', 'Azeri', 'Slovene', 'Kyrgyz', 'Bislama', 'Italian', 'Catalan', 'Modern Greek', 'Bengali', 'Belarusian', 'Malay', 'Persian', 'Tuvaluan', 'Tongan', 'Nepali', 'Mandarin', 'Khmer', 'Urdu', 'Croatian', 'Dzongkha', 'Malagasy', 'Mongolian', 'Somali', 'Tajik', 'Sinhala', 'Maltese', 'Dari', 'Marshallese', 'Maldivian', 'Luxembourgish', 'Latvian'}


In [284]:
# First langauge such as Chinese (Taiwan), Spanish (Mexico), etc must be Chinese and Spanish respectively. 
twitch['language'].loc[twitch['language']== 'Chinese (Hong Kong SAR)'] = 'Chinese'
twitch['language'].loc[twitch['language']== 'Chinese (Taiwan)'] = 'Chinese'
twitch['language'].loc[twitch['language']== 'Spanish (Mexico)'] = 'Spanish'
twitch['language'].loc[twitch['language']== 'Portuguese (Brazil)'] = 'Portuguese'

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_8684/1858301311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitch['language'].loc[twitch['language']== 'Chinese (Hong Kong SAR)'] = 'Chinese'
/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_8684/1858301311.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitch['language'].loc[twitch['language']== 'Chinese (Taiwan)'] = 'Chinese'
/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_8684/1858301311.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [285]:
# We will still need to fix languages such as ltailans and Italian. 
# Also broadband do not have language named Czech and Polish. Moreover, Mandarin must be changed to Chinese. 
broad['language'].loc[broad['language'] == 'Catalan'] = 'Catalans'
broad['language'].loc[broad['language'] == 'Italian'] = 'Italians'
broad['language'].loc[broad['language'] == 'Hungarian'] = 'Hungarians'
broad['language'].loc[broad['language'] == 'Modern Greek'] = 'Greek'
broad['language'].loc[broad['language'] == 'Malay'] = 'Malaysian'
broad['language'].loc[broad['language'] == 'Mandarin'] = 'Chinese'

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_8684/761493659.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broad['language'].loc[broad['language'] == 'Catalan'] = 'Catalans'
/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_8684/761493659.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  broad['language'].loc[broad['language'] == 'Italian'] = 'Italians'
/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_8684/761493659.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

In [286]:
# Twitch data contains the language called Polish, but we do not have polish in our sample. 
# For that reason we will assign German to Polish in Twitch data. 
twitch['language'].loc[twitch['language']== 'Polish'] = 'German'

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_8684/3775273233.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitch['language'].loc[twitch['language']== 'Polish'] = 'German'


In [287]:
# Finally, the remaining language in bro_lan - twi_lan should be converted to Other category. 
# Put the set_prize - set_twitch language category to Other category. 
wi_lan = set(twitch['language'].unique())
bro_lan = set(broad['language'].unique())

# Create a function that converts rest of them to "Other"
def change_names(textstr):
    if textstr in list(bro_lan - wi_lan):
        return 'Other'
    else:
        return textstr

# Run the function on the dataframe  
broad['language']=broad['language'].apply(change_names)

In [288]:
# now that we have fixed that, let us check again if we still need any fix. 
# Note that broadband data do not have Czech in the sample. So we will drop it when merging twitch and broadband data. 
# Moreover, All languages and American Sign Language will also be dropped when merging. 
twi_lan = set(twitch['language'].unique())
bro_lan = set(broad['language'].unique())
print(twi_lan - bro_lan)
print(bro_lan - twi_lan)

{'Czech', 'All languages', 'American Sign Language'}
set()


In [289]:
# Now that twitch data and broadband data is cleaned, we will now clean the Prize data. However, doing this is a difficult task. 
# First of all, prize data contain location information that is just not formatted in a consistent format. 
# Second of all, the data does not contain the game name. So we will do that as well. 
# Thirdly, I would like to create a seperate column for year and month using startdata as date. 

prize = pd.read_csv('prize_money.csv')
game_nam = pd.read_csv('game_names.csv')

# First let us merge two data using GameID as a key. 
prize = pd.merge(prize, game_nam, on='GameId')

In [290]:
# Create year and month column
month_names = {
    1: 'january', 2: 'february', 3: 'march', 4: 'april',
    5: 'may', 6: 'june', 7: 'july', 8: 'august',
    9: 'september', 10: 'october', 11: 'november', 12: 'december'
}

prize["date"]= pd.to_datetime(prize['StartDate'], format='%m/%d/%y', errors='coerce')
# One data contains NA value. 
prize[prize['date'].isna()]

,TournamentId,GameId,TournamentName,StartDate,EndDate,Location,Teamplay,TotalUSDPrize,GameName,date
47592,56072,785,IVRL Onward S1 Championship,0202-05-07,5/16/22,Online,1,8000.0,Onward,NaT


In [291]:
# We will fix NA value and run the code again. 

prize.loc[47592, 'StartDate'] = '5/7/20'

prize["date"]= pd.to_datetime(prize['StartDate'], format='%m/%d/%y', errors='coerce')
prize[prize['date'].isna()]

,TournamentId,GameId,TournamentName,StartDate,EndDate,Location,Teamplay,TotalUSDPrize,GameName,date


In [292]:
# Make the year and month column and drop StartDate and EndDate. 
prize['year'] = prize['date'].dt.year
prize['month'] = prize['date'].dt.month.map(month_names)
prize = prize.drop(['StartDate','EndDate'], axis=1)
prize.tail()

,TournamentId,GameId,TournamentName,Location,Teamplay,TotalUSDPrize,GameName,date,year,month
63106,1395,229,WCG 2001 (FIFA 2001),"Seoul, South Korea",0,35000.0,FIFA 2001,2001-12-05,2001,december
63107,13904,414,WCG Challenge 2000 (FIFA 2000),"Seoul, South Korea",0,50000.0,FIFA 2000,2000-10-08,2000,october
63108,8027,365,"Descent 3 $50,000 Tournament","Las Vegas, Nevada, USA",0,50000.0,Descent 3,1999-08-26,1999,august
63109,62654,848,PGL Season 2 (TA),"Atlanta, Georgia, USA",0,15000.0,Total Annihilation,1998-05-28,1998,may
63110,62650,847,PGL Season 1 (C&C: RA),"Seattle, Washington, USA",0,15000.0,Command & Conquer: Red Alert,1998-01-30,1998,january


In [293]:
# Now we would like to clean the location data. To do this, we will make things consistent by converting all into a lowercase.
prize["Location"] = prize['Location'].str.lower()

# Our strategy is to take the unique value of each location data and convert them into a dataframe. 
# Then find a corresponding country data using a library. 

location = pd.DataFrame({'location': list(prize['Location'].str.lower().unique())})
location.head()

,location
0,online
1,"barcelona, spain"
2,italy
3,poland
4,taiwan


In [294]:
# This function will take the locaiton data and find the corresponding country. 
# Running this might take a while. 

from geopy.geocoders import Nominatim

def get_country(location):
    geolocator = Nominatim(user_agent="location_identifier")
    location_info = geolocator.geocode(location, language='en', timeout=10)
    
    if location_info:
        return location_info.address.split(",")[-1].strip()
    else:
        return "Not Found"

# We will assign the country and save it to location.csv so that we do not need to run it again. 
location['country'] = location['location'].apply(get_country)
location.to_csv('uni_loc1.csv', index=False)


In [295]:
# Download the file
location = pd.read_csv('/Users/yuyaogawa/Documents/Home Work/Research with Dr. Ward/uni_loc1.csv')

In [296]:
# The function has assigned 116 unique country to the location data. However, some of them are not accurate. 
pd.DataFrame(location['country'].unique()).count()

# I attempted to find an efficient way to fix this, but manually fixing it turned out to be the easiest solution. 
# I have created a csv file that is already fixed and combined them with the prize data, and we will download it here.

prize = pd.read_csv('clean_prize_money.csv')

In [297]:
game = pd.DataFrame(prize['GameName'].unique()).sort_values(by=0).reset_index(drop=True)

# We will create a column containing the first word of the game name. 
game[1] = game[0].str.split().str.get(0)

# Seems like the games that share the same first word are essentially the equivalent games. 

# Create a column called name
game['name'] = 0
game.at[0, 'name'] = 'ARMS'
# The following code will give the same name of the game if the first word is the same. 
for i in range(len(game)-1):
    if game.iloc[i][1] == game.iloc[i+1][1]:
        game.at[i+1, 'name'] = game.iloc[i]['name']
    else:
        game.at[i+1, 'name'] = game.iloc[i+1][0]

# Result is successful. 
game = game.drop(1, axis=1)
game

,0,name
0,ARMS,ARMS
1,Acceleration of SUGURI 2,Acceleration of SUGURI 2
2,After-H,After-H
3,Age of Empires,Age of Empires
4,Age of Empires II,Age of Empires
...,...,...
493,chessarena.com,chessarena.com
494,eFootball,eFootball
495,iRacing.com,iRacing.com
496,osu!,osu!


In [298]:
# Now that we have identified equivalent games, we will merge game column and prize column using 0 as a key. 
# We will merge the game name data with twitch as well.

prize = pd.merge(prize, game, left_on='GameName', right_on=0)
prize = prize.drop(['GameName', 0], axis = 1)
prize = prize.rename(columns={'name': 'GameName'})


In [299]:
twitch = pd.merge(twitch, game, left_on='GameName', right_on=0)
twitch = twitch.drop(['GameName', 0], axis = 1)
twitch = twitch.rename(columns={'name': 'GameName'})


In [300]:
# Before merging this dataset with broadband data, we need to aggregate prize data as well. 
# We will do so by taking the average and std conditional on country, GameName, and year. 

prize = prize.groupby(['country', 'year', 'GameName', 'language']).agg({
    'TotalUSDPrize': ['mean', 'std']
}).reset_index()

# Rename columns for clarity
prize.columns = [f'{col[0]}_{col[1]}' if col[1] != '' else col[0] for col in prize.columns]

In [301]:
prize.tail()

,country,year,GameName,language,TotalUSDPrize_mean,TotalUSDPrize_std
5877,Vietnam,2023,Arena of Valor,Vietnamese,334113.066667,143746.193049
5878,Vietnam,2023,CrossFire,Vietnamese,187500.000000,159099.025767
5879,Vietnam,2023,League of Legends,Vietnamese,48785.300000,2283.530639
5880,Vietnam,2023,PLAYERUNKNOWN'S BATTLEGROUNDS Mobile,Vietnamese,49982.520000,1329.106190
5881,Vietnam,2023,VALORANT,Vietnamese,23035.723333,15240.143640


In [302]:
broad.tail()

,Entity,Code,Year,Fixed broadband subscriptions (per 100 people),country,language
3419,Zimbabwe,ZWE,2017,1.269804,Zimbabwe,English
3420,Zimbabwe,ZWE,2018,1.349014,Zimbabwe,English
3421,Zimbabwe,ZWE,2019,1.331353,Zimbabwe,English
3422,Zimbabwe,ZWE,2020,1.298439,Zimbabwe,English
3423,Zimbabwe,ZWE,2021,1.283851,Zimbabwe,English


In [303]:
# Finally, the prize data is fixed. 
# Now we will merge broadband data and prize data using country and year as a key. 
# broad contains irrelevant columns so we will drop them.
broad = broad.drop(['Entity', 'Code'], axis = 1)
broad = broad.rename(columns={'Year':'year'})
prize = pd.merge(prize, broad, on = ['country', 'year'])

In [304]:
# We will keep the language_y which is consistent with twitch data. 
prize = prize.drop('language_x', axis=1)
prize = prize.rename(columns={'language_y':'language'})
prize.head()

,country,year,GameName,TotalUSDPrize_mean,TotalUSDPrize_std,Fixed broadband subscriptions (per 100 people),language
0,Albania,2021,Counter-Strike: Condition Zero,70052.95,NaN,19.595476,Other
1,Argentina,2004,StarCraft: Brood War,200.00,NaN,1.403920,Spanish
2,Argentina,2005,StarCraft: Brood War,450.00,NaN,2.371923,Spanish
3,Argentina,2009,Counter-Strike,3660.00,NaN,8.629920,Spanish
4,Argentina,2012,Call of Duty,765.79,NaN,12.272450,Spanish


In [305]:
# When we merge twitch data with the rest of them, we would like to merge using year and country as a key. 
# But the problem is that twitch data is monthly data and broadband is an yearly data. 
# For above reason, we will have to aggregate twitch data into an yearly data. 
# To do this, we will take the average of view times, stream times, etc, conditional on games, year, and language. 
# In aggregated twitch data, We will create a column called average and standard deviation. 

agg_twitch = twitch.groupby(['year', 'language', 'GameName']).agg({
    'watch_time_min': ['mean', 'std'],
    'stream_time_min': ['mean', 'std'],
    'peak_viewers': ['mean', 'std'],
    'peak_channels': ['mean', 'std'],
    'streamers': ['mean', 'std']
}).reset_index()

agg_twitch.columns = [f'{col[0]}_{col[1]}' if col[1] != '' else col[0] for col in agg_twitch.columns]


In [306]:
print(len(agg_twitch), len(prize), len(broad))

8575 4985 3424


In [307]:
# Now that we have aggregated data for twitch, we can merge prize data and twitch using language, year, and GameName as a key.
# Before that, we will drop the country column from prize data. 
prize = prize.drop('country', axis=1)
final_data = pd.merge(prize, agg_twitch, on=['year', 'language','GameName'], how='inner')
final_data 


,year,GameName,TotalUSDPrize_mean,TotalUSDPrize_std,Fixed broadband subscriptions (per 100 people),language,watch_time_min_mean,watch_time_min_std,stream_time_min_mean,stream_time_min_std,peak_viewers_mean,peak_viewers_std,peak_channels_mean,peak_channels_std,streamers_mean,streamers_std
0,2016,League of Legends,4500.000,2121.320344,16.606285,Spanish,1.743176e+08,4.345332e+07,2.176265e+06,351554.126196,34692.916667,23285.835388,133.500000,18.193405,5762.833333,1140.938277
1,2016,League of Legends,9984.136,2553.054175,30.367228,Spanish,1.743176e+08,4.345332e+07,2.176265e+06,351554.126196,34692.916667,23285.835388,133.500000,18.193405,5762.833333,1140.938277
2,2016,Super Mario Party,60.700,45.290841,16.606285,Spanish,2.063937e+05,1.292273e+05,7.013684e+03,3748.487414,408.894737,278.322134,4.736842,1.147079,52.157895,21.279263
3,2016,Super Mario Party,242.815,282.734917,30.367228,Spanish,2.063937e+05,1.292273e+05,7.013684e+03,3748.487414,408.894737,278.322134,4.736842,1.147079,52.157895,21.279263
4,2017,League of Legends,15536.250,21713.421936,17.802399,Spanish,2.045839e+08,3.755448e+07,3.580170e+06,843938.005285,36751.250000,19308.253276,188.083333,23.971416,10295.000000,1896.588516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,2021,Age of Empires,1696.130,904.952089,19.830288,Vietnamese,2.695200e+04,2.986481e+04,2.070000e+03,2107.655220,31.350000,27.917689,1.550000,0.510418,3.150000,1.926956
1461,2021,Arena of Valor,124822.330,NaN,19.830288,Vietnamese,1.870000e+03,2.501767e+03,1.200000e+02,151.789328,13.500000,17.398276,1.333333,0.516398,9.000000,4.427189
1462,2021,FIFA 03,8637.885,35.305842,19.830288,Vietnamese,1.647273e+03,1.078787e+03,1.690909e+02,119.453300,8.090909,3.534248,1.090909,0.301511,4.454545,2.296242
1463,2021,League of Legends,64958.850,44357.467776,19.830288,Vietnamese,1.039080e+05,3.712875e+05,2.805000e+03,2869.446123,653.600000,2187.279021,2.450000,1.190975,37.300000,31.158508


In [309]:
final_data.to_csv('final_data.csv', index=False)